# Spark assignment

Taratukhin Oleg, C41323

Initial env setup:

In [20]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pathlib import Path
import emoji

spark = SparkSession.builder \
    .appName("itmo spark assignment") \
    .master("local[*]") \
    .getOrCreate()

spark

In [41]:
!cd data && ls
!rm -rf results/task6/

PATH_TO_DATA = Path("data/")
PATH_TO_RESULTS = Path("results/task6/")

followers.parquet              posts_api.json
followers_posts_api_final.json posts_likes.parquet
followers_posts_likes.parquet


Utilities:

In [22]:
def save_as_json(df: pyspark.sql.dataframe.DataFrame, name: str):
    df.write \
        .format("json") \
        .save(str(PATH_TO_RESULTS / name))

In [30]:
followers_posts_likes = spark \
    .read \
    .parquet(str(PATH_TO_DATA / "followers_posts_likes.parquet"))

followers_posts_likes.limit(10).toPandas()

,itemType,ownerId,itemId,likerId
0,post,1567,3324,2767
1,post,1567,3324,254280
2,post,1567,3324,553430
3,post,1567,3324,905379
4,post,1567,3324,2411502
5,post,1567,3324,5274303
6,post,1567,3324,50038414
7,post,1567,3324,84207602
8,post,1567,3324,145422426
9,post,1567,3331,4601081


## Task 6

Find fans/lovers. The idea is, if the user likes another posts, and its is not mutual, then the first fan/lover

In [42]:
fans = followers_posts_likes.alias("df1") \
    .join(followers_posts_likes.alias("df2"),
          ((col("df1.ownerId") == col("df2.likerId")) &
           (col("df1.ownerId") == col("df2.likerId"))), 'leftanti') \
    .distinct() \
    .orderBy("df1.ownerId") \
    .select("ownerId", "likerId")

save_as_json(fans, "task6_fans.json")

fans.limit(10).toPandas()

,ownerId,likerId
0,1632,3216106
1,1632,386930170
2,1632,19022461
3,1632,11187963
4,1632,195477126
5,1632,8403484
6,1632,118099
7,1632,34932945
8,1632,557124343
9,1632,153154650
